# Cargar

In [1]:
#FILE='CIS_Microsoft_Intune_for_Windows_10_Benchmark_v2.0.0-1'
#FILE='CIS_Microsoft_Intune_for_Windows_10_Benchmark_v2.0.0'
FILE='CIS_Microsoft_Windows_Server_2008_(non-R2)_Benchmark_v3.3.1_ARCHIVE'
#FILE='CIS_Microsoft_Windows_Server_2012_(non-R2)_Benchmark_v3.0.0_FINAL_UPDATE'
#FILE='CIS_Microsoft_Windows_Server_2016_Benchmark_v2.0.0'
#FILE='CIS_Microsoft_Windows_Server_2016_STIG_Benchmark_v2.0.0' 

In [2]:
import pandas as pd
import numpy as np
import re
from deep_translator import GoogleTranslator

In [3]:
df=pd.read_excel('../../output/Windows/clean/'+FILE+'_clean.xlsx')

In [4]:
df.fillna('',inplace=True)

## Traducción

Para la traducción de los textos se utiliza el paquete `GoogleTranslator` de `deep_translator`. La traducción a español puede tomar un par de minutos, además el paquete solo permite traducir 5.000 caracteres peticiones, por lo que se debe asegurar que ningún texto supere este límite.

In [8]:
# UNCOMMENT TO TRANSLATE TO SPANISH
"""
df['Parametro']=df['Parametro'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))
df['Descripción']=df['Descripción'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))
df['Procedimiento de implementación']=df['Procedimiento de implementación'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))
df['Procedimiento de verificación']=df['Procedimiento de verificación'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))

"""

"\ndf['Parametro']=df['Parametro'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))\ndf['Descripción']=df['Descripción'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))\ndf['Procedimiento de implementación']=df['Procedimiento de implementación'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))\ndf['Procedimiento de verificación']=df['Procedimiento de verificación'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))\n\n"

### Unir configuraciones

Por último se procede a unir las configuraciones en cada columna, el **Procedimiento de implementación** se une con la configuración de **Implementación** y el **Procedimiento de verificación** se une con la configuración de **Verificación** excepto en el caso de que la configuración de **Verificación** sea vacía, en ese caso se une con la configuración de **Implementación**.


In [5]:
df['Procedimiento de implementación']=df['Procedimiento de implementación'] + '\n' + df['Config implementación']

In [6]:
df['Procedimiento de verificación']=df[['Procedimiento de verificación', 'Config verificación', 'Config implementación']].apply(
    lambda x:
        x['Procedimiento de verificación'] + '\n' + x['Config implementación'] if x['Config verificación'] == ''
        else
        x['Procedimiento de verificación'] + '\n' + x['Config verificación']
    , axis=1)

In [7]:
df.drop(columns=['Config implementación', 'Config verificación'], inplace=True)

### Guardar en Excel en diferentes hojas

Se guarda el excel en diferentes hojas, cada hoja corresponde a una seccion de controles al segundo nivel, esto es 1.1. 1.2. 1.3. etc.

In [8]:
#Extraer secciones finales
df['Sección']=df['IDCIS'].apply(lambda x: re.findall(r'(\d{1,2}\.\d{1,2})', x)[0])

In [9]:
column_width = 50

with pd.ExcelWriter('../../output/Windows/'+FILE+'_es'+'.xlsx', engine='openpyxl') as writer:
    for i in df['Sección'].unique():
        df_seccion=df[df['Sección']==i]
        df_seccion.drop(columns=['Sección'], inplace=True)
        df_seccion.to_excel(writer, index=False, sheet_name= 'Sección '+i)

        # Access the XlsxWriter workbook and worksheet objects
        workbook  = writer.book
        worksheet = writer.sheets['Sección '+i]

        # Set the column width for each column
        for i, column in enumerate(df_seccion.columns):
            column_letter = chr(ord('A') + i)
            worksheet.column_dimensions[column_letter].width = column_width

/tmp/ipykernel_16385/534308970.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seccion.drop(columns=['Sección'], inplace=True)
/tmp/ipykernel_16385/534308970.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seccion.drop(columns=['Sección'], inplace=True)
/tmp/ipykernel_16385/534308970.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seccion.drop(columns=['Sección'], inplace=True)
/tmp/ipykernel_16385/534308970.py:6: 